This notebook implements a finitary version of the approach to the probabilities of conditionals from Bas van Fraassen's 1976 paper "Probabilities of conditionals" (https://link.springer.com/chapter/10.1007/978-94-010-1853-1_10) (cf. Section 5 of http://paolosantorio.net/KS-NASSLLI2018.pdf).

# Outline

### 1. Preliminaries

### 2. Semantics

### 3. Probability

#### 3.1 Fair die case

#### 3.2 Biased die case

### 4. Modified semantics

#### 4.1. McGee-style counterexample to Modus Ponens

# 1. Preliminaries

In [967]:
def perms(urn, num_to_draw = None):
    """
    Returns all permutations of subsets of urn of size num_to_draw (default is len(urn))
    """
    if num_to_draw == None:
        num_to_draw = len(urn)

    return _perms(urn,num_to_draw,[])

def _perms(urn, num_to_draw, sequence):

    if num_to_draw == 1:
        for w in urn:
            yield tuple(sequence + [w])

    if num_to_draw > 1:
        for w in urn:
            yield from _perms([v for v in urn if not v==w], num_to_draw-1, sequence + [w])

In [968]:
for seq in perms(['x','y','z']):
    print(seq)

('x', 'y', 'z')
('x', 'z', 'y')
('y', 'x', 'z')
('y', 'z', 'x')
('z', 'x', 'y')
('z', 'y', 'x')


In [969]:
def permutations_of_ne_subsets(urn):
    """
    Returns all permutations of nonempty subsets of worlds
    """

    for n in range(len(urn)+1):

        yield from perms(urn,n)

In [970]:
for seq in permutations_of_ne_subsets(['x','y','z']):
    print(seq)

('x',)
('y',)
('z',)
('x', 'y')
('x', 'z')
('y', 'x')
('y', 'z')
('z', 'x')
('z', 'y')
('x', 'y', 'z')
('x', 'z', 'y')
('y', 'x', 'z')
('y', 'z', 'x')
('z', 'x', 'y')
('z', 'y', 'x')


# 2. Semantics

A *model* is a pair $\mathcal{M}=(W,V)$ of a nonempty set $W$ and a propositional varluation $V:\mathsf{Prop}\to \wp(W)$.

Let $W'$ be the set of all permutations of nonempty subsets of $W$. We define truth of a formula $\varphi$ at a permutation $\langle w_1,\dots,w_n\rangle\in W'$ as follows:

1. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash p$ iff $w_1\in V(p)$;

2. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \neg\varphi$ iff $\mathcal{M},\langle w_1,\dots,w_n\rangle\nvDash \varphi$;

3. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi\wedge\psi$ iff $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi$ and $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi$;

4. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi\vee\psi$ iff $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi$ or $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi$;

5. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi > \psi$ iff $\mathrm{min}(\{i \in\mathbb{N}\mid \mathcal{M},\langle w_i,\dots,w_n\rangle\vDash \varphi \})\subseteq \{i \in\mathbb{N}\mid \mathcal{M},\langle w_i,\dots,w_n\rangle\vDash \psi \}$,

where $\langle w_i,\dots,w_n\rangle$ is the final segment of $\langle w_1,\dots,w_n\rangle$ beginning with $w_i$.

We call $\{\sigma\in W'\mid \mathcal{M},\sigma\vDash\varphi\}$ the *extension* of $\varphi$.

In [971]:
def main_connective(formula):
    """
    Returns the main connective of the formula (or "prop" if s is a propositional variable) and arguments to main connective (or propositional variable)
    """

    formula = formula.replace(" ", "")
        
    if len(formula) == 1:
        return "prop", formula

    if formula.startswith("~"):
        formula = formula.removeprefix("~")
        return "~", formula

    if formula.startswith("("):
            
        leftparen = 0
        rightparen = 0
        slength = len(formula)

        for n in range(slength):

            if formula[n] == "(":
                leftparen +=1

            if formula[n] == ")":
                rightparen +=1

            if leftparen - rightparen == 1:
                
                if formula[n] == "^":
                    return "^", formula[1:n], formula[n+1:-1]

                if formula[n] == "v":
                    return "v", formula[1:n], formula[n+1:-1]
                
                if formula[n] == '>':
                    return '>', formula[1:n], formula[n+1:-1]     

    return None

In [972]:
def extension(formula, sequences, valuation):
    """
    Given a formula, a set of sequences of worlds, and a valuation mapping each proposition letter to a set of worlds,
    returns the extension of the formula as a set of sequences
    """
    
    main_connec = main_connective(formula)

    if main_connec[0] == 'prop':

        return set([sequence for sequence in sequences if sequence[0] in valuation[main_connec[1]]])

    if main_connec[0] == '~':
        inner_ext = set(extension(main_connec[1], sequences,valuation))
        return set([w for w in sequences if not w in inner_ext])

    if main_connec[0] == '^':
        return extension(main_connec[1],sequences,valuation).intersection(extension(main_connec[2],sequences,valuation))

    if main_connec[0] == 'v':
        return extension(main_connec[1],sequences,valuation).union(extension(main_connec[2],sequences,valuation))

    if main_connec[0] == '>':

        ant_ext = extension(main_connec[1], sequences, valuation)
        cons_ext = extension(main_connec[2], sequences, valuation)

        ext = [seq for seq in sequences]

        for sequence in sequences:
            for idx, world in enumerate(sequence):
                if sequence[idx::] in ant_ext:
                    if not sequence[idx::] in cons_ext:

                        ext.remove(sequence)
                        break
                        
                    else:
                        break

        return set(ext)

In [973]:
seqs = list(permutations_of_ne_subsets(['x','y','z']))
print(seqs)

[('x',), ('y',), ('z',), ('x', 'y'), ('x', 'z'), ('y', 'x'), ('y', 'z'), ('z', 'x'), ('z', 'y'), ('x', 'y', 'z'), ('x', 'z', 'y'), ('y', 'x', 'z'), ('y', 'z', 'x'), ('z', 'x', 'y'), ('z', 'y', 'x')]


In [974]:
extension('p',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('z',),
 ('z', 'x'),
 ('z', 'x', 'y'),
 ('z', 'y'),
 ('z', 'y', 'x')}

In [975]:
extension('q',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('y',),
 ('y', 'x'),
 ('y', 'x', 'z'),
 ('y', 'z'),
 ('y', 'z', 'x')}

In [976]:
extension('(p>q)',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('y',),
 ('y', 'x'),
 ('y', 'x', 'z')}

In [977]:
extension('(q>p)',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('z',),
 ('z', 'x'),
 ('z', 'x', 'y')}

In [978]:
extension('(p>(q>p))',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('y',),
 ('y', 'x'),
 ('y', 'x', 'z'),
 ('y', 'z'),
 ('y', 'z', 'x'),
 ('z',),
 ('z', 'x'),
 ('z', 'x', 'y')}

# 3. Probability of formulas

Given a finite model $\mathcal{M}=(W,V)$, we encode a probability distribution as a dictionary that assigns to each $w\in W$ a number in $[0,1]$ such that the probabilities sum to 1.

In [979]:
mu = dict()
mu['x'] = .4
mu['y'] = .35
mu['z'] = .25

Next we assign probabilities to permutations of $W$. Note: we do not assign probabilities to permutations of proper subsets of $W$, since this will not be needed in the following.

We take the probability of a permutation of $W$ to be the probability of drawing that permutation via sampling without replacement from $W$ according to the given probability distribution.

In [980]:
def prob_without_replacement(sequence,world_prob_dist):
    """
    Given a sequence of worlds and a probability distribution over worlds,
    returns the probability of the sequence under sampling without replacement
    """

    world = sequence[0]

    if len(sequence) == 1:
        return world_prob_dist[world]

    new_world_prob_dist = dict()

    for w in world_prob_dist.keys():
        if not w == sequence[0]:

            new_world_prob_dist[w] =  world_prob_dist[w] /(1-world_prob_dist[world])

    return world_prob_dist[world] * prob_without_replacement(sequence[1:],new_world_prob_dist)

In [981]:
prob_without_replacement(['x','y','z'], mu)

0.2333333333333334

In [982]:
def sequence_prob_dist(world_prob_dist):
    """
    Given a probability distribution over worlds,
    returns the probability distribution over permutations of the set of worlds
    given by sampling without replacement
    """

    sequence_prob_dist = dict()

    for sequence in perms(world_prob_dist.keys()):

        sequence_prob_dist[sequence] =  prob_without_replacement(sequence,world_prob_dist)

    return sequence_prob_dist

In [983]:
sequence_prob_dist(mu)

{('x', 'y', 'z'): 0.2333333333333334,
 ('x', 'z', 'y'): 0.16666666666666674,
 ('y', 'x', 'z'): 0.2153846153846154,
 ('y', 'z', 'x'): 0.13461538461538458,
 ('z', 'x', 'y'): 0.1333333333333333,
 ('z', 'y', 'x'): 0.11666666666666663}

Given a model $\mathcal{M}=(W,V)$ and probabilities encoded by $\mu:W\to[0,1]$, let $\mu^+:\wp(W)\to [0,1]$ be defined as usual by $\mu^+(A)=\underset{w\in A}{\sum}\mu(w)$.

Then we take the probability of a formula $\varphi$ to be $\mu^+(\{\sigma\in permutations(W)\mid \mathcal{M},\sigma\vDash\varphi\})$.

In [984]:
def prob_of_formula(formula, world_prob_dist, valuation):
    """
    Given a formula, probability distribution over worlds, and propositional valuation,
    returns the probability of the formula
    """

    worlds = world_prob_dist.keys()

    sequences = list(permutations_of_ne_subsets(worlds))

    ext = extension(formula,sequences,valuation)

    seq_prob_dist = sequence_prob_dist(world_prob_dist)

    full_perms_ext = [seq for seq in perms(worlds) if seq in ext]

    return sum([seq_prob_dist[seq] for seq in full_perms_ext])

## 3.1. Fair die case

In [985]:
fair_die = dict()

for n in range(1,7):
    fair_die[n] = 1/6

print(fair_die)


{1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666}


In [986]:
val = {'l': set({1,2,3}), 'h': set({4,5,6}), 'e': set({2,4,6})}

In [987]:
prob_of_formula('(l>e)', fair_die, val)

0.3333333333333324

In [988]:
prob_of_formula('(e^l)', fair_die, val) / prob_of_formula('l', fair_die, val)

0.33333333333333365

In [989]:
prob_of_formula('(h>e)', fair_die, val)

0.6666666666666646

In [990]:
prob_of_formula('(e^h)', fair_die, val) / prob_of_formula('h', fair_die, val)

0.6666666666666669

In [991]:
prob_of_formula('((l>e)^(h>e))', fair_die ,val) 

0.22222222222222168

By contrast, according to Ciardelli and Ommunndsen (2022, Sec. 5) (https://onlinelibrary.wiley.com/doi/abs/10.1111/nous.12437), $(l>e)\wedge (h>e)$ is equivalent to $e$ and hence should have probability .5. By this reasoning, the probability of $(l>e)\wedge (h>e)$ should also be .5 in the following biased die case. 

## 3.2. Biased die case

In [992]:
biased_die = dict()

biased_die[1] = 4/24
biased_die[2] = 1/24
biased_die[3] = 4/24
biased_die[4] = 6/24
biased_die[5] = 4/24
biased_die[6] = 5/24

In [993]:
prob_of_formula('e', biased_die, val)

0.5000000000000006

In [994]:
prob_of_formula('(l>e)', biased_die, val)

0.11111111111111105

In [995]:
prob_of_formula('(e^l)', biased_die, val) / prob_of_formula('l', biased_die, val)

0.11111111111111112

In [996]:
prob_of_formula('(h>e)', biased_die, val)

0.7333333333333321

In [997]:
prob_of_formula('(e^h)', biased_die, val) / prob_of_formula('h', biased_die, val)

0.733333333333335

In [998]:
prob_of_formula('((l>e)^(h>e))', biased_die, val)

0.08148148148148146

As noted above, the reasoning of Ciardelli and Ommunndsen (2022, Sec. 5) implies that the probability of $(l>e)\wedge (h>e)$ should be .5 in this case.

# 4. Modified semantics

We now give a modified semantics that validates $p> (q > p)$.

The semantics is intended only for conditionals $\varphi>\psi$ where $\varphi$ is Boolean. Allowing conditionals in $\varphi$ makes things more complicated.

The new clause for $>$ is:

$\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi > \psi$ iff $\mathrm{min}(\{i \in\mathbb{N}\mid \mathcal{M},\langle w_i,\dots,w_n\rangle\vDash \varphi \})\subseteq \{i \in\mathbb{N}\mid \mathcal{M},\langle w_i,\dots,w_n\rangle +\varphi \vDash \psi \}$ where $\langle w_i,\dots,w_n\rangle +\varphi$ is the sequence obtained from $\langle w_i,\dots,w_n\rangle$ by dropping those $w_j$ such that $\mathcal{M},\langle w_j,\dots,w_n\rangle\nvDash \varphi$.


In [999]:
def extension2(formula, sequences, valuation):
    """
    Given a formula, a set of sequences of worlds, and a valuation mapping each proposition letter to a set of worlds,
    returns the extension of the formula as a set of sequences according to the modified semantics
    """
    
    main_connec = main_connective(formula)

    if main_connec[0] == 'prop':

        return set([sequence for sequence in sequences if sequence[0] in valuation[main_connec[1]]])

    if main_connec[0] == '~':
        inner_ext = set(extension2(main_connec[1], sequences,valuation))
        return set([w for w in sequences if not w in inner_ext])

    if main_connec[0] == '^':
        return extension2(main_connec[1],sequences,valuation).intersection(extension2(main_connec[2],sequences,valuation))

    if main_connec[0] == 'v':
        return extension2(main_connec[1],sequences,valuation).union(extension2(main_connec[2],sequences,valuation))

    if main_connec[0] == '>':

        ant_ext = extension2(main_connec[1], sequences, valuation)
        cons_ext = extension2(main_connec[2], sequences, valuation)

        ext = [seq for seq in sequences]

        for sequence in sequences:
            
            for idx, world in enumerate(sequence):

                final_segment = sequence[idx::]
                
                if sequence[idx::] in ant_ext:

                    restricted_sequence = tuple()

                    for idx2, world2 in enumerate(final_segment):

                        if final_segment[idx2::] in ant_ext:
                            restricted_sequence = restricted_sequence + (world2,)

                    if not restricted_sequence in cons_ext:

                        ext.remove(sequence)
                        break
                        
                    else:
                        break

        return set(ext)

In [1000]:
def prob_of_formula2(formula, world_prob_dist, valuation):
    """
    Given a formula, probability distribution over worlds, and propositional valuation,
    returns the probability of the formula according to the modified semantics
    """

    worlds = world_prob_dist.keys()

    sequences = list(permutations_of_ne_subsets(worlds))

    ext = extension2(formula,sequences,valuation)

    seq_prob_dist = sequence_prob_dist(world_prob_dist)

    full_perms_ext = [seq for seq in perms(worlds) if seq in ext]

    return sum([seq_prob_dist[seq] for seq in full_perms_ext])

## 4.1. McGee-style counterexample to modus ponens

This example is from Section 4.2 of Paolo Santorio's (2021) "Trivializing Informational Consequence" (http://paolosantorio.net/tic.pdf), based on McGee's (1985) "A Counterexample to Modus Ponens" (https://www.pdcnet.org/jphil/content/jphil_1985_0082_0009_0462_0471).

In [1001]:
seq = list(permutations_of_ne_subsets([1,2,3,4,5,6]))
val2 = {'e': set({2,4,6}), 't': set({2}), 'f': set({4}), 's': set({6})}

print(len(seq))
print(len(extension('(e>((~t ^ ~f)>s))', seq, val2))) # The formula is not true at all sequences according to the semantics of Section 2
print(len(extension2('(e>((~t ^ ~f)>s))', seq, val2))) # But it is true at all sequences according to the modified semantics


1956
1134
1956


In [1002]:
prob_of_formula2('(e>((~t ^ ~f)>s))', fair_die, val2)

0.9999999999999968

In [1003]:
prob_of_formula2('(e^(e>((~t ^ ~f)>s)))', fair_die, val2)

0.4999999999999985

In [1004]:
prob_of_formula2('((~t ^ ~f)>s)', fair_die, val2)

0.24999999999999936